### Purpose: To obtain object statistics from impages imported by user

*To do List:*
4. Extract Basic Statistics:
Euler number
Image Centroids
On Both Object Areas and distances from object centroids to the image centroid
Min 
Max
Mean
Median
Variance
10-bin histogram 

*Decided to do a global threshold and a local threshold * - do more research into the difference between these two things

*Step 1: Import the necessary packages*

In [17]:
import sys
sys.path.append('/Users/HawleyHelm/Desktop/CHE_545_546/Rockstar-Lifestyle')

In [18]:
from PIL import Image, ImageFilter, ImageEnhance
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import scipy
from scipy.ndimage import gaussian_filter
from RockstarLifestyle import ObjectStats
from skimage import filters
from skimage.morphology import watershed
from skimage.feature import peak_local_max
from skimage import measure
from skimage import morphology
from scipy import ndimage
from skimage import segmentation
from RockstarLifestyle import preprocessing

*Step 2: User controlled inputs*

In [29]:
im_contrast_value = 2

*Step 3: Image Import*

In [19]:
im =  Image.open("Test_Photo_fromMike.png")

*Step 4: Splitting image into red, green, blue values*

In [41]:
r,g,b = im.split()

In [45]:
contrast_image = preprocessing.image_contrast(g, 2)

__Step 4: Otsu Global Threshold*__ 

In [50]:
contrast_image_arr = np.array(contrast_image)

In [51]:
thresh = filters.threshold_otsu(contrast_image_arr)
thresh

97

*Step 5: Obtaining the Binary Mask of the image*

In [52]:
binary = image > thresh

*Step 6: Finding the eight connection items in the binary mask*

__Step 4: Local Threshold__

Question for me: how do you figure out what block size to use in local thresholding? - seems like i could easily introduce bias here

In [53]:
block_size = 15
adaptive_thresh = filters.threshold_local(contrast_image_arr, block_size, offset=8)
binary_adaptive = image < adaptive_thresh

*Step 5: Find all eight connection items in the binary mask*

In [54]:
distance = ndimage.distance_transform_edt(binary)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((3, 3)), labels=binary)

In [55]:
markers = morphology.label(local_maxi)
markers[~binary] = -1

In [56]:
labels_rw = segmentation.random_walker(binary, markers)

*Step 5: Object labeling with local Threshold*

In [57]:
distance = ndimage.distance_transform_edt(binary_adaptive)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((3, 3)), labels=binary_adaptive)

In [58]:
markers = morphology.label(local_maxi)
markers[~binary_adaptive] = -1

In [ ]:
labels_rw_adaptive = segmentation.random_walker(binary_adaptive, markers)

In [ ]:
plt.imshow(labels_rw_adaptive) 

In [ ]:
fig = plt.figure()
ax1 = plt.subplot(221)
ax2 = plt.subplot(222)
ax3 = plt.subplot(223)

ax1.imshow(labels_rw) 
ax2.imshow(labels_rw_adaptive)
ax3.imshow(g)

fig.tight_layout()

*Step 6: Extract Basic Statistics*

In [ ]:
properties = measure.regionprops(labels_rw)
[prop.perimeter for prop in properties] 

*Step 6a: 10-bin histogram*

In [ ]:
plt.hist(labels_rw, bins=10, density = True, cumulative = True)

In [ ]:
*Step 6b: Min, Ma*